In [4]:
import sys
sys.argv = sys.argv + ['-data' 'scripts/sandbox/data/multi30k.atok.low.pt' '-save_model' 'trained' '-save_mode best' '-proj_share_weight' '-label_smoothing']

In [7]:
here = __file__

%cd {os.path.basename(here)}

import argparse
import math
import time

import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from tqdm import tqdm

from instanceseg.models.transformerconstants import Constants
from instanceseg.models.transformer import Transformer
from instanceseg.utils.optim import ScheduledOptim
from scripts.sandbox.attention_is_all_you_need_dataset import TranslationDataset, paired_collate_fn

NameError: name '__file__' is not defined

In [3]:
def cal_performance(pred, gold, smoothing=False):
    ''' Apply label smoothing if needed '''

    loss = cal_loss(pred, gold, smoothing)

    pred = pred.max(1)[1]
    gold = gold.contiguous().view(-1)
    non_pad_mask = gold.ne(Constants.PAD)
    n_correct = pred.eq(gold)
    n_correct = n_correct.masked_select(non_pad_mask).sum().item()

    return loss, n_correct


def cal_loss(pred, gold, smoothing):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.1
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        non_pad_mask = gold.ne(Constants.PAD)
        loss = -(one_hot * log_prb).sum(dim=1)
        loss = loss.masked_select(non_pad_mask).sum()  # average later
    else:
        loss = F.cross_entropy(pred, gold, ignore_index=Constants.PAD, reduction='sum')

    return loss
